# Sparse Search

In [1]:
import Data.Array

In [2]:
import qualified Data.Sequence as S

In [3]:
{-# LANGUAGE ScopedTypeVariables #-}

In [4]:
type StringArray i = Array i String

type Interval i = (i, i)
type IntervalQueue i = S.Seq (Interval i)

sparseSearch :: forall i. (Integral i, Ix i, Show i) => String -> StringArray i -> Maybe i
sparseSearch x xs =
    let
        stopWord = ""

        popInterval :: IntervalQueue i -> (Interval i, IntervalQueue i)
        popInterval intervals = (S.index intervals 0, S.drop 1 intervals)

        modifiedBinary :: IntervalQueue i -> Maybe i
        modifiedBinary intervals
            | left > right =
                if S.null intervals' then
                    Nothing
                else
                    modifiedBinary intervals'
            | otherwise =
                if x' == stopWord && x /= stopWord then
                    modifiedBinary $ intervals' S.|> leftHalf S.|> rightHalf
                else
                    case compare x x' of
                        EQ -> Just middle
                        LT -> modifiedBinary $ intervals' S.|> leftHalf
                        GT -> modifiedBinary $ intervals' S.|> rightHalf
            where
                ((left, right), intervals') = popInterval intervals

                middle = left + (right - left) `div` 2
                leftHalf = (left, (middle - 1))
                rightHalf = ((middle + 1), right)

                x' = xs ! middle
    in
        modifiedBinary $ S.singleton $ bounds xs

In [5]:
xs = ["at", "", "", "", "ball", "", "", "car", "", "", "dad", "", ""]
xs' = listArray (0, length xs - 1) xs

"ball" `sparseSearch` xs'

Just 4

## Tests

In [6]:
import Test.QuickCheck

In [7]:
import Data.List (nub, sort)

In [8]:
testSparseSearchRandomized :: [String] -> Bool
testSparseSearchRandomized xs =
    let
        xs' = sort xs
        
        haystack = listArray (1, length xs') xs'
        needles = nub xs'
        
        checkIndex :: (String, Maybe Int) -> Bool 
        checkIndex (needle, index) =
            maybe False (\i -> haystack ! i == needle) index
    in
        all checkIndex $ zip needles $ map (`sparseSearch` haystack) needles

In [9]:
quickCheck testSparseSearchRandomized

+++ OK, passed 100 tests.